Modified code from Hasanzadeh et al., (2019) ([original code link](https://github.com/navidhasanzadeh/BPPPG), [paper link](https://ieeexplore.ieee.org/document/8938751)).

In [7]:
# install special packages
!pip install biosppy
!pip install hrv-analysis
# !apt update && apt install -y libsm6 libxext6
# !apt-get install -y libxrender-dev

from pyampd.ampd import find_peaks
import numpy as np
import scipy
from biosppy.signals import bvp
from scipy import io
from hrvanalysis import get_time_domain_features
from hrvanalysis import get_frequency_domain_features
import h5py
import pandas as pd

In [8]:
freq = 125
debug = False
dataset_dir = '../../../datasets/MIMIC-II/'
dataset_name = 'hasanzadeh19'
filter_name = 'hasanzadeh19'
alg_name = 'hasandazeh19'
datapath = dataset_dir + dataset_name + '.h5'
feature_interval = 1 # extract every n beats
ppg_idx = 0
ecg_idx = 1
abp_idx = 2

save_name = 'features_' + dataset_name + '_' + filter_name + '_' + alg_name + '.csv'
data = h5py.File(datapath, 'r')
keys = np.array(list(data.keys()))[np.arange(0, len(data.keys()))]

In [9]:
all_features = pd.DataFrame()
for i in range(len(keys)):
    subject_features = pd.DataFrame()
    print("subject = {}".format(i))
    if dataset_name == 'vitaldb':
        seg_idxs = np.arange(0, data[keys[i]].shape[0])
        segs = np.random.choice(seg_idxs, size=np.minimum([10], [data[keys[i]].shape[0]]), replace=False)
    else:
        segs = np.arange(0, data[keys[i]].shape[0])
    for r in range(len(segs)):
        PPG = data[keys[i]][segs[r], ppg_idx, :]
        BP = data[keys[i]][segs[r], abp_idx, :]
        details = [keys[i].astype(int),segs[r],2,3,np.max(BP), np.min(BP), 6,7,8]
        try:    
            # Filtering the signal        
            # the library uses filtfilt (=> Phase = 0)
            ppg_filtered_low = bvp.st.filter_signal(signal=PPG,ftype='butter',band='lowpass',order=6,frequency=10,sampling_rate=freq)[0]
            if (int(15 * 1000/freq)-1)%2 > 0:
                medfilt_param = int(15 * 1000/freq)-1
            else:
                medfilt_param = int(15 * 1000/freq)
            ppg_filtered = ppg_filtered_low - scipy.signal.medfilt(ppg_filtered_low, medfilt_param)                 
            # Calculating the peaks and minimums of the signal (using AMPD)
            onsets = find_peaks(-1 * ppg_filtered, scale=200)
            peaks = find_peaks(ppg_filtered, scale=200)
            nn_intervals_list  = np.diff(onsets) * (1000/freq)
            time_domain_features = list(get_time_domain_features(nn_intervals_list).values())
            freq_domain_features = list(get_frequency_domain_features(nn_intervals_list).values())

            HR = 60/(np.median(np.diff(onsets))/freq)
            PPG = np.array(PPG)
            record_features = []
            for onset in list(range(1,len(onsets)-1,feature_interval)):
                try:
                    segment_features = []
                    for win_num in range(0,feature_interval):
                        ppg_window = ppg_filtered[onsets[onset + win_num]:onsets[onset + win_num + 1]]
                        ppg_window = ppg_window - np.min(ppg_window)
                        ppg_window = ppg_window / np.max(ppg_window)                          
                        window_features = []
                        ignore_flag = False
                        # Detecting maximum of the window and dividing the Pulse into two ascending and descending parts
                        ppg_window_systolic = np.argmax(ppg_window)
                        ppg_window_ascending = ppg_window[:ppg_window_systolic]
                        ppg_window_descending = ppg_window[ppg_window_systolic:]
                        # Fitting polynomials to each part and calculating their first and second derivatives
                        ppg_window_ascending_x = np.linspace(0,ppg_window_systolic,len(ppg_window_ascending))
                        ppg_window_descending_x = np.linspace(ppg_window_systolic,len(ppg_window),len(ppg_window_descending))
                        asc_weights = np.ones(np.shape(ppg_window_ascending))
                        asc_weights[int(0.25*len(asc_weights)):int(0.75*len(asc_weights))] = 1
                        asc_weights[int(0.4*len(asc_weights)):int(0.6*len(asc_weights))] = 1
                        des_weights = np.ones(np.shape(ppg_window_descending))
                        des_weights[int(0.25*len(des_weights)):int(0.75*len(des_weights))] = 1
                        des_weights[int(0.4*len(des_weights)):int(0.6*len(des_weights))] = 1
                        des_weights[int(0.8*len(des_weights)):int(1*len(des_weights))] = 1
                        ppg_window_ascending_poly5 = np.polyfit(ppg_window_ascending_x ,ppg_window_ascending,5,w=asc_weights)
                        ppg_window_descending_poly7 = np.polyfit(ppg_window_descending_x,ppg_window_descending,7,w=des_weights)
                        ppg_window_ascending_poly5_der = np.poly1d(np.polyder(ppg_window_ascending_poly5))
                        ppg_window_descending_poly7_der = np.poly1d(np.polyder(ppg_window_descending_poly7))
                        ppg_window_descending_poly7_der2 = np.poly1d(np.polyder(ppg_window_descending_poly7,2))
                        ppg_window_ascending_poly5_der_values = ppg_window_ascending_poly5_der(ppg_window_ascending_x)
                        ppg_window_descending_poly7_der_values = ppg_window_descending_poly7_der(ppg_window_descending_x)
                        ppg_window_descending_poly7_der2_values = ppg_window_descending_poly7_der2(ppg_window_descending_x)
                        # calculating the location of Max. slope point
                        ppg_maxslope = np.argmax(ppg_window_ascending_poly5_der_values)
                        # calculating the location of Diastolic peak, Dicrotic Notch and inflection point based on the method presented in our paper
                        ppg_window_descending_poly7_der_roots = np.sort(np.array([np.real(root) for root in np.roots(ppg_window_descending_poly7_der) if ~np.iscomplex(root)]))
                        ppg_window_descending_poly7_der_roots = np.array([root for root in ppg_window_descending_poly7_der_roots if (root>ppg_window_systolic*1.25 and root<0.95*len(ppg_window))])
                        ppg_window_descending_poly7_der_roots_der2_values = ppg_window_descending_poly7_der2(ppg_window_descending_poly7_der_roots)
                        if len(ppg_window_descending_poly7_der_roots)==0:
                            ppg_no_dia = True
                        else:
                            dia_candidates = []
                            for dia_index, dia_candidate in enumerate(ppg_window_descending_poly7_der_roots):
                                if ppg_window_descending_poly7_der_roots_der2_values[dia_index]<0:
                                    dia_candidates.append(dia_candidate)
                            if len(dia_candidates)>0:
                                ppg_dia = dia_candidates[int(len(dia_candidates)/2)]
                                ppg_no_dia = False
                            else:
                                ppg_no_dia = True
                        if ppg_no_dia == True:
                            ppg_des_der2_minimums = scipy.signal.find_peaks(-1*ppg_window_descending_poly7_der2_values)[0] + ppg_window_systolic
                            dia_candidates = np.array([root for root in ppg_des_der2_minimums if (root>ppg_window_systolic*1.25 and root<0.95*len(ppg_window))])
                            if len(dia_candidates)>0:
                                ppg_dia = dia_candidates[0]            
                            else:
                                ppg_dia = ppg_window_systolic + int(len(ppg_window_descending)/2)
                        # find dicrotic notch
                        ppg_des_der2_maximums = scipy.signal.find_peaks(ppg_window_descending_poly7_der2_values)[0] + ppg_window_systolic
                        dicrotic_candidates = np.array([root for root in ppg_des_der2_maximums if (root>ppg_window_systolic*1.2 and root<0.995*ppg_dia)])
                        if len(dicrotic_candidates)>0:
                            ppg_dic = dicrotic_candidates[0]           
                        else:
                            ppg_dic = ppg_des_der2_maximums[0]
                        # find inflection point
                        ppg_window_descending_poly7_der2_roots = np.sort(np.array([np.real(root) for root in np.roots(ppg_window_descending_poly7_der2) if ~np.iscomplex(root)]))
                        inf_candidates = np.array([root for root in ppg_window_descending_poly7_der2_roots if (root>ppg_dic and root<ppg_dia)])
                        if len(inf_candidates)>0:
                            ppg_inf = inf_candidates[0]
                        else:
                            ignore_flag = True
                            ppg_inf = (ppg_dia + ppg_dic)/2

                        ppg_dia = int(ppg_dia)
                        ppg_dic = int(ppg_dic)
                        ppg_inf = int(ppg_inf)

                        # When point detection is done, extract the features
                        window_features.extend(details)
                        meanBP = (1*details[4] + 2*details[5]) / 3
                        window_features.extend([0])
                        window_features.extend([meanBP])                                        
                        ppg_window_main = PPG[onsets[onset + win_num]:onsets[onset + win_num + 1]]
                        ppg_window_main = np.array(ppg_window_main)            
                        ppg_dc = np.mean(PPG)
                        ppg_window_main_rev = ppg_window_main[::-1]
                        try:
                            ppg_last_min = len(ppg_window_main)- scipy.signal.find_peaks(-1*ppg_window_main_rev)[0][0]
                        except:
                            ppg_last_min = len(ppg_window_main) -1
                        ppg_last_min = (ppg_window_main[0]+ppg_window_main[ppg_last_min])/2    
                        #
                        ppg_ac = np.median(PPG[peaks]) - np.median(PPG[onsets])
                        # modified Normalized Pulse Volume
                        mNPV1 = ppg_ac / (ppg_dc)
                        mNPV2 = ppg_ac / (ppg_dc+ppg_ac)
                        # Area related features
                        A1 = np.trapz(ppg_window_main[0:ppg_maxslope])
                        A2 = np.trapz(ppg_window_main[ppg_maxslope:ppg_window_systolic])
                        A3 = np.trapz(ppg_window_main[ppg_window_systolic:ppg_dic])
                        A4 = np.trapz(ppg_window_main[ppg_dic:ppg_inf])
                        A5 = np.trapz(ppg_window_main[ppg_inf:ppg_dia])    
                        A6 = np.trapz(ppg_window_main)
                        S1 = A1/A2
                        S2 = A1/A3
                        S3 = A1/A4
                        S4 = A1/A5
                        S5 = A1/A6
                        S6 = A2/A3
                        S7 = A2/A4
                        S8 = A2/A5
                        S9 = A2/A6
                        S10 = A3/A4
                        S11 = A3/A5
                        S12 = A3/A6
                        S13 = A4/A5
                        S14 = A4/A6
                        S15 = A5/A6
                        S16 = (A1+A2+A3+A4)/(A5+A6)
                        #
                        NA1 = np.trapz(ppg_window[0:ppg_maxslope])
                        NA2 = np.trapz(ppg_window[ppg_maxslope:ppg_window_systolic])
                        NA3 = np.trapz(ppg_window[ppg_window_systolic:ppg_dic])
                        NA4 = np.trapz(ppg_window[ppg_dic:ppg_inf])
                        NA5 = np.trapz(ppg_window[ppg_inf:ppg_dia])    
                        NA6 = np.trapz(ppg_window)
                        NS1 = NA1/NA2
                        NS2 = NA1/NA3
                        NS3 = NA1/NA4
                        NS4 = NA1/NA5
                        NS5 = NA1/NA6
                        NS6 = NA2/NA3
                        NS7 = NA2/NA4
                        NS8 = NA2/NA5
                        NS9 = NA2/NA6
                        NS10 = NA3/NA4
                        NS11 = NA3/NA5
                        NS12 = NA3/NA6
                        NS13 = NA4/NA5
                        NS14 = NA4/NA6
                        NS15 = NA5/NA6
                        NS16 = (NA1+NA2+NA3+NA4)/(NA5+NA6)
                        # Reflection Indices
                        RI1 = (ppg_window_main[ppg_maxslope] - ppg_last_min) / (ppg_window_main[ppg_window_systolic] - ppg_last_min)
                        RI2 = (ppg_window_main[ppg_dic] - ppg_last_min) / (ppg_window_main[ppg_window_systolic] - ppg_last_min)
                        RI3 = (ppg_window_main[ppg_inf] - ppg_last_min) / (ppg_window_main[ppg_window_systolic] - ppg_last_min)
                        RI4 = (ppg_window_main[ppg_dia] - ppg_last_min) / (ppg_window_main[ppg_window_systolic] - ppg_last_min)
                        # LASI
                        SI1 = details[2]/(ppg_dia-ppg_window_systolic)
                        SI2 = details[2]/(ppg_dic-ppg_window_systolic)
                        SI3 = details[2]/(ppg_inf-ppg_window_systolic)
                        SI4 = details[2]/(ppg_dia-ppg_maxslope)
                        SI5 = details[2]/(ppg_window_systolic-ppg_maxslope)
                        SI6 = details[2]/(ppg_dia-ppg_inf)
                        # Non-linear features
                        ln_mNPV1 = np.log(mNPV1)
                        ln_mNPV2 = np.log(mNPV2)
                        exp_mNPV1 = np.exp(mNPV1)
                        exp_mNPV2 = np.exp(mNPV2)
                        ln_HR = np.log(HR)
                        exp_HR = np.exp(HR)
                        ln_HRmNPV1 = np.log(mNPV1 * HR)
                        ln_HRmNPV2 = np.log(mNPV2 * HR)
                        ln_RI1 = np.log(RI1)
                        ln_RI2 = np.log(RI2)
                        ln_RI3 = np.log(RI3)
                        ln_RI4 = np.log(RI4)
                        window_features.extend([ppg_dc,ppg_ac,mNPV1,mNPV2,A1,A2,A3,A4,A5,A6,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,NA1,NA2,NA3,NA4,NA5,NA6,NS1,NS2,NS3,NS4,NS5,NS6,NS7,NS8,NS9,NS10,NS11,NS12,NS13,NS14,NS15,NS16,RI1
                                                ,RI2,RI3,RI4,SI1,SI2,SI3,SI4,SI5,SI6,ln_mNPV1,ln_mNPV2,exp_mNPV1,exp_mNPV2,ln_HRmNPV1,ln_HRmNPV2,ln_RI1,ln_RI2,ln_RI3,ln_RI4,HR,ln_HR,exp_HR])
                        window_features.extend([ppg_maxslope,ppg_window_systolic,ppg_dic,ppg_dia,ppg_inf])
                        window_features.extend(time_domain_features)
                        window_features.extend(freq_domain_features)
                        segment_features.append(window_features)
                    record_features.append(np.mean(segment_features,0))            
                except:                
#                         print('^', end='')
                    pass
            if len(np.median(record_features,0))>0:
                subject_features = pd.concat([subject_features, pd.DataFrame(np.median(record_features,0)).T.rename({0:r})])
#                 print(subject_features)
#                 print("")
#                 print("++")
        except:                
            print("**")
            pass
    if len(subject_features) > 0:
        all_features = pd.concat([all_features, pd.DataFrame(subject_features).groupby(0).mean().reset_index()])

subject = 0


/opt/conda/lib/python3.8/site-packages/scipy/signal/spectral.py:1961: UserWarning: nperseg = 256 is greater than input length  = 25, using nperseg = 25
  warnings.warn('nperseg = {0:d} is greater than input length '
/opt/conda/lib/python3.8/site-packages/scipy/signal/spectral.py:1961: UserWarning: nperseg = 256 is greater than input length  = 28, using nperseg = 28
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-9-f93b70363424>:156: RuntimeWarning: divide by zero encountered in double_scalars
  S4 = A1/A5
<ipython-input-9-f93b70363424>:160: RuntimeWarning: divide by zero encountered in double_scalars
  S8 = A2/A5
<ipython-input-9-f93b70363424>:163: RuntimeWarning: divide by zero encountered in double_scalars
  S11 = A3/A5
<ipython-input-9-f93b70363424>:165: RuntimeWarning: divide by zero encountered in double_scalars
  S13 = A4/A5
<ipython-input-9-f93b70363424>:179: RuntimeWarning: divide by zero encountered in double_scalars
  NS4 = NA1/NA5
<ipython-inpu

subject = 1


<ipython-input-9-f93b70363424>:153: RuntimeWarning: divide by zero encountered in double_scalars
  S1 = A1/A2
<ipython-input-9-f93b70363424>:159: RuntimeWarning: invalid value encountered in double_scalars
  S7 = A2/A4
<ipython-input-9-f93b70363424>:160: RuntimeWarning: invalid value encountered in double_scalars
  S8 = A2/A5
<ipython-input-9-f93b70363424>:176: RuntimeWarning: divide by zero encountered in double_scalars
  NS1 = NA1/NA2
<ipython-input-9-f93b70363424>:182: RuntimeWarning: invalid value encountered in double_scalars
  NS7 = NA2/NA4
<ipython-input-9-f93b70363424>:183: RuntimeWarning: invalid value encountered in double_scalars
  NS8 = NA2/NA5
<ipython-input-9-f93b70363424>:205: RuntimeWarning: divide by zero encountered in log
  ln_mNPV1 = np.log(mNPV1)
<ipython-input-9-f93b70363424>:206: RuntimeWarning: divide by zero encountered in log
  ln_mNPV2 = np.log(mNPV2)
<ipython-input-9-f93b70363424>:211: RuntimeWarning: divide by zero encountered in log
  ln_HRmNPV1 = np.log(m

subject = 2


...

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWar

subject = 3349
subject = 3350
subject = 3351
subject = 3352
subject = 3353
subject = 3354


/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWar

subject = 3355
subject = 3356


/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3415: RankWarning: Polyfit may be poorly conditioned
  await eval(code_obj, self.user_global_ns, self.user_ns)


In [13]:
df = pd.DataFrame(all_features)
df = df.drop([0,1,2,3,6,7,8,9], axis=1)
df.replace([np.inf, -np.inf],np.nan, inplace=True)
df.fillna(df.mean(),inplace=True)
df = df.reset_index(drop=True)
df = df.drop(df[df[4] < 90].index)
df = df.drop(df[df[5] < 40].index)
df = df.drop(df[df[10] < 50].index)
df = df.drop(df[df[4] > 200].index)
df = df.drop(df[df[5] > 100].index)
df = df.drop(df[df[10] > 140].index)
df = df.drop([10], axis=1)

col_names_1 = ['SBP','DBP']
# manually extracted features
col_names_2 = ['ppg_dc','ppg_ac','mNPV1','mNPV2','A1','A2','A3','A4','A5','A6',
             'S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14','S15','S16',
             'NA1','NA2','NA3','NA4','NA5','NA6','NS1','NS2','NS3','NS4','NS5','NS6','NS7',
             'NS8','NS9','NS10','NS11','NS12','NS13','NS14','NS15','NS16',
             'RI1','RI2','RI3','RI4','SI1','SI2','SI3','SI4','SI5','SI6',
             'ln_mNPV1','ln_mNPV2','exp_mNPV1','exp_mNPV2','ln_HRmNPV1','ln_HRmNPV2',
             'ln_RI1','ln_RI2','ln_RI3','ln_RI4','HR','ln_HR','exp_HR',
             'ppg_maxslope','ppg_window_systolic','ppg_dic','ppg_dia','ppg_inf']
# time domain features
col_names_3 = ['Mean_NNI', 'SDNN', 'SDSD', 'NN50', 'pNN50', 'NN20', 'pNN20', 'RMSSD', 'Median_NN',
               'Range_NN', 'CVSD', 'CV_NNI', 'Mean_HR', 'Max_HR', 'Min_HR', 'STD_HR']
# freq domain features
col_names_4 = ['LF', 'HF', 'VLF', 'LH/HF ratio', 'LFnu', 'HFnu', 'Total_Power']

col_names = col_names_1.copy()
col_names.extend(col_names_2)
col_names.extend(col_names_3)
col_names.extend(col_names_4)
len(col_names)

df.columns = col_names
df.to_csv('../../results/features/' + save_name, index=False)

In [12]:
# df = pd.DataFrame(all_features)
# df.to_csv('../../results/features/' + save_name, index=False)

In [1]:
!pwd

/root/wearablebp_benchmarks/classical_ml/BPPPG
